# 1. 라이브러리 설치 및 호출

In [0]:
from typing import Union
import pandas as pd
import pyspark
from pyspark import sql as pysql
from pyspark.sql import SparkSession

# 2. 사용자 정의 함수: 유틸 함수들

## 2-1. 데이터베이스 조회

In [0]:
def show_databases() -> None:
    host = "20.196.145.211"
    port = "5432"
    user = "postgres"
    password = "asdASD123!@#"

    jdbc_url = f"jdbc:postgresql://{host}:{port}/postgres?user={user}&password={password}"
    query = "(SELECT datname FROM pg_database WHERE datistemplate = false) AS databases"
    df = spark.read.jdbc(url=jdbc_url, table=query, properties={"driver": "org.postgresql.Driver"})
    df.show(truncate=False)

## 2-2. 스키마 조회

In [0]:
def show_schemas(database: str = "postgres") -> None:
    host = "20.196.145.211"
    port = "5432"
    user = "postgres"
    password = "asdASD123!@#"

    jdbc_url = f"jdbc:postgresql://{host}:{port}/{database}?user={user}&password={password}"
    query = "(SELECT schema_name FROM information_schema.schemata) AS schemas"
    df = spark.read.jdbc(url=jdbc_url, table=query, properties={"driver": "org.postgresql.Driver"})
    df.show(truncate=False)

## 2-3. 테이블 조회

In [0]:
def show_tables(schema: str, database: str = "postgres") -> None:
    host = "20.196.145.211"
    port = "5432"
    user = "postgres"
    password = "asdASD123!@#"

    jdbc_url = f"jdbc:postgresql://{host}:{port}/{database}?user={user}&password={password}"
    query = f"""
        (SELECT table_name
         FROM information_schema.tables
         WHERE table_type='BASE TABLE'
           AND table_schema = '{schema}'
         ORDER BY table_name) AS tables
    """
    df = spark.read.jdbc(url=jdbc_url, table=query, properties={"driver": "org.postgresql.Driver"})
    df.show(truncate=False)

## 2-4. 테이블 속 데이터 조회

In [0]:
def show_datas(schema: str, table: str, database: str = "postgres") -> None:
    host = "20.196.145.211"
    port = "5432"
    user = "postgres"
    password = "asdASD123!@#"

    # DB 접속 URL
    jdbc_url = f"jdbc:postgresql://{host}:{port}/{database}"

    # 스키마.테이블 지정
    dbtable = f"{schema}.{table}"
    df = (
        spark.read.format("jdbc")
        .option("url", jdbc_url)
        .option("user", user)
        .option("password", password)
        .option("driver", "org.postgresql.Driver")
        .option("dbtable", dbtable)
        .load()
    )
    df.show(truncate=False)

In [0]:
def insert_data(
    data: Union[pd.DataFrame, pyspark.sql.DataFrame],
    schema: str,
    table: str,
    mode: str = "overwrite"
) -> None:
    # 데이터프레임 생성
    # spark = SparkSession.builder.appName("DataFrameExample").getOrCreate()
    columns = list(data.columns)
    if isinstance(data, pd.DataFrame):
        data = spark.createDataFrame(data, schema=columns)
    
    # 인증 정보 하드코딩
    jdbc_user = "postgres"
    jdbc_pass = "asdASD123!@#"

    # 연결 문자열 하드코딩
    host = "20.196.145.211"
    port = "5432"
    jdbc_url = f"jdbc:postgresql://{host}:{port}/postgres"

    data.write.format("jdbc").mode(mode).options(
        url=jdbc_url,
        user=jdbc_user,
        password=jdbc_pass,
        dbtable=f"{schema}.{table.lower()}",
        # # overwrite일 때 DROP 후 CREATE가 아니라, TRUNCATE TABLE로 데이터만 비움
        # truncate=True,
        # 대량 적재 튜닝: INSERT할 때 한 번에 몇 행씩 묶어서 넣을지
        batchsize=1000,
        # 스키마 적재 튜닝: INSERT시 JDBC는 setString() 함수로 text 자동 매핑
        # 이 옵션을 사용하면 unspecified로 넘겨 PostgreSQL이 자동 캐스팅
        stringtype="unspecified"
    ).save()
    return

# 3. 함수 실행 가이드

## 3-1. 예제 데이터 생성

In [0]:
# 입력 예제로 사용할 데이터프레임
test_df = pd.DataFrame({
    "name": ["alice", "bob", "charlie"],
    "age": [11, 22, 33]
})

## 3-2. 함수 실행 예제

In [0]:
# 데이터베이스 목록을 조회하는 코드
show_databases()

# 스키마 목록을 조회하는 코드
show_schemas()

# 특정 스키마 아래의 테이블 목록을 조회하는 코드
show_tables(schema="test_data")

# 특정 테이블의 데이터를 조회하는 코드
show_datas(schema="test_data", table="test_table")

# 특정 테이블에 데이터를 삽입하는 코드
# mode 종류: overwrite, append, ignore, error
insert_data(data=test_df, schema="test_data", table="test_table", mode="overwrite")

+--------+
|datname |
+--------+
|postgres|
+--------+

+------------------+
|schema_name       |
+------------------+
|public            |
|gold              |
|silver            |
|bronze            |
|test_data         |
|test              |
|information_schema|
|pg_catalog        |
|pg_toast          |
+------------------+

+----------+
|table_name|
+----------+
|test_table|
+----------+

+----------------+-----------+
|name            |age        |
+----------------+-----------+
|ccccccccccharlie|3333333333 |
|aaaaalice       |11111111111|
|bbbbbbbbbob     |2222222    |
|ccccccccccharlie|3333333333 |
|bbbbbbbbbob     |2222222    |
|aaaaalice       |11111111111|
|bbbbbbbbbob     |2222222    |
|aaaaalice       |11111111111|
|ccccccccccharlie|3333333333 |
|aaaaalice       |11111111111|
|bbbbbbbbbob     |2222222    |
|ccccccccccharlie|3333333333 |
+----------------+-----------+



In [0]:
# 특정 테이블의 데이터를 조회하는 코드
show_datas(schema="test_data", table="test_table")

+----------------+-----------+
|name            |age        |
+----------------+-----------+
|bbbbbbbbbob     |2222222    |
|ccccccccccharlie|3333333333 |
|aaaaalice       |11111111111|
+----------------+-----------+



---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-8993352249731298>, line 208
    204     print("▶ 연차 수요(타겟) vs 배정합:", exp_demand, "/", X.sum(axis=0))
    205     print("▶ 총합:", X.sum(), " (N =", N, ")")
--> 208 dummy()

File <command-8993352249731298>, line 2, in dummy()
      1 def dummy():
----> 2     import math, numpy as np, pandas as pd, networkx as nx
      3     from io import StringIO
      5     # ---------------- 입력 ----------------

ModuleNotFoundError: No module named 'networkx'